In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
model = keras.models.load_model('normal_model.h5')
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 100, 100, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 100, 100, 32)      320       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 32)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 50, 50, 64)        18496     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 64)        0         
_________________________________________________________________
reshape (Reshape)            (None, 25, 1600)          0         
_________________________________________________________________
dense1 (Dense)               (None, 25, 64)            102464

In [3]:
characters = [' ', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '@', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), num_oov_indices=0, mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

max_length = 3

In [5]:
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

def test(string):
    fname = f'../generated_test/{string}.png'
    img = tf.io.read_file(fname)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [100, 100])
    img = tf.transpose(img, perm=[1, 0, 2])
    result = model.predict(np.array([img]))
    return decode_batch_predictions(result)[0].replace('[UNK]', '*').replace(' ', '')

In [47]:
# be st us er @g ma il .c om

#email = 'vla dis lav 199 9@g ma il. com'
#email = 'dan il sem eno v@y an dex .ru'
email = 'che bur ash ka@ hot mai l.c om'

assert(len(email.split()) == 8)
email_recognized = []
for part in email.split():
    email_recognized.append(test(part))

print(' '.join(email_recognized))

che bur asb ka@ hot nai l.* om
